In [15]:
# importación de pandas para gráfico de tablas y seaborn para mapa de calor de correlación
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# definimos que pandas imprima todas las columnas al mostrar un dataframe
pd.set_option('display.max_columns', None)


In [16]:
# importanción de anime y users (tablas relativamente cortas)
anime = pd.read_csv('anime_cleaned.csv')
users = pd.read_csv('users_cleaned.csv')


In [17]:
# la imporanción de animelists es un poco más lenta
# por lo que se realiza en otra celda
animelists = pd.read_csv('animelists_cleaned.csv')


In [4]:
animelists_copy = animelists.copy()

In [14]:
print(animelists_copy.head())
print(animelists_copy.columns)
print(animelists_copy.shape)

              username  anime_id  my_watched_episodes  my_score  my_status  \
6740075  ----phoebelyn       949                    6         0          2   
6740012  ----phoebelyn       135                   75         0          2   
6740014  ----phoebelyn       153                   45         0          2   
6740020  ----phoebelyn       245                   43        10          2   
6740026  ----phoebelyn       486                   13         0          2   

         my_rewatching  my_rewatching_ep  
6740075            0.0                 0  
6740012            0.0                 0  
6740014            0.0                 0  
6740020            0.0                 0  
6740026            0.0                 0  
Index(['username', 'anime_id', 'my_watched_episodes', 'my_score', 'my_status',
       'my_rewatching', 'my_rewatching_ep'],
      dtype='object')
(19946135, 7)


## preprocesamiento de datos

In [6]:
animelists_copy = animelists_copy.sort_values(by='username')
animelists_copy = animelists_copy.dropna(subset=['username'])
animelists_copy = animelists_copy.drop(columns=[
    'my_start_date', 'my_finish_date', 'my_tags', 'my_last_updated'
    ])

In [7]:
#Eliminar todos los usuarios que no han completado un anime, quedandose solo con los usuarios que si han completado un anime
animelists_copy = animelists_copy[animelists_copy["my_status"] == 2]
#Dropear todas laanimelists_copys columnas menos 'anime_id' y 'username'
gen_clusters = animelists_copy[["anime_id", "username"]]

In [8]:
print("users columns", users.columns)
print("anime columns", anime.columns)

users columns Index(['username', 'user_id', 'user_watching', 'user_completed', 'user_onhold',
       'user_dropped', 'user_plantowatch', 'user_days_spent_watching',
       'gender', 'location', 'birth_date', 'access_rank', 'join_date',
       'last_online', 'stats_mean_score', 'stats_rewatched', 'stats_episodes'],
      dtype='object')
anime columns Index(['anime_id', 'title', 'title_english', 'title_japanese',
       'title_synonyms', 'image_url', 'type', 'source', 'episodes', 'status',
       'airing', 'aired_string', 'aired', 'duration', 'rating', 'score',
       'scored_by', 'rank', 'popularity', 'members', 'favorites', 'background',
       'premiered', 'broadcast', 'related', 'producer', 'licensor', 'studio',
       'genre', 'opening_theme', 'ending_theme', 'duration_min',
       'aired_from_year'],
      dtype='object')


In [9]:
#Dropear del dataset 'anime' todas las columnas excepto 'anime' y 'genero'
anime = anime.drop(columns=[
    'image_url', 'background', 'related', 'premiered', 'broadcast',
    'opening_theme', 'ending_theme', 'title_english', 'title_japanese', 'title_synonyms',
    'airing', 'aired_string', 'licensor'
    ])
anime = anime[["anime_id", "genre"]]
print("anime: ",anime.shape)

anime:  (6668, 2)


In [10]:
#Filtrar columnas user_name, genre, y edad
users['age'] = 2018 - users['birth_date'].str[:4].astype(int)
users = users[["username", "gender", "age"]]
print("users: ", users.shape)

users:  (108711, 3)


In [11]:
gen_clusters = gen_clusters.merge(anime, on="anime_id", how="inner")
gen_clusters = gen_clusters.merge(users, on="username", how="inner")

In [19]:
gen_clusters = gen_clusters[['genre','gender','age']]
print(gen_clusters.shape)
print(gen_clusters.columns)

(19946135, 3)
Index(['genre', 'gender', 'age'], dtype='object')


In [21]:
# Primero buscamos todos los generos y productores que existen

generos = []
for lista in gen_clusters['genre']:
    if type(lista) == str:
        lista = lista.split(', ')
        for genero in lista:
            if genero not in generos:
                generos.append(genero)


['Action', 'Comedy', 'Drama', 'Mecha', 'Military', 'Sci-Fi', 'Shounen', 'Space', 'Game', 'Supernatural', 'Adventure', 'Fantasy', 'Magic', 'School', 'Slice of Life', 'Psychological', 'Romance', 'Shoujo', 'Police', 'Seinen', 'Historical', 'Samurai', 'Super Power', 'Harem', 'Mystery', 'Music', 'Cars', 'Sports', 'Demons', 'Parody', 'Thriller', 'Ecchi', 'Horror', 'Vampire', 'Josei', 'Martial Arts', 'Shoujo Ai', 'Kids', 'Dementia', 'Yuri', 'Shounen Ai', 'Hentai', 'Yaoi']


In [22]:
gen_clusters_copy = gen_clusters.copy()
# Creamos una columna nueva en el dataframe anime por cada gener
for genero in generos:
    if type(genero) == str:
        gen_clusters_copy[genero] = gen_clusters_copy['genre'].str.contains(genero).astype(int)

ValueError: cannot convert float NaN to integer

## análisis de clusters

In [23]:
# imports para realizar el análisis de la pregunta 2 con distintos métodos de clustering
from sklearn.cluster import KMeans
from sklearn import datasets
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
import numpy as np

